In [18]:
import torch
import torch.utils.data as data
import os
import pickle
import numpy as np
import nltk
from PIL import Image
import os.path
import random
from torch.autograd import Variable
import torch.nn as nn
import math
import cv2
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
import argparse
import torch
import torch.nn as nn
import numpy as np
import os
import pickle
from torch.autograd import Variable 
import math
import csv

In [41]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1), 
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1), 
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64 , 128 , 3 , stride = 2 , padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 , 256 , 3 , stride = 2 , padding = 1),
            nn.ReLU(),
            nn.Conv2d(256 , 512 , 3 , stride = 2 , padding = 1),
            nn.ReLU(),
            nn.Conv2d(512 , 1024 , 2),
            nn.ReLU(),
            nn.Conv2d(1024 , 2048 , 1),
            nn.ReLU(),
            nn.Conv2d(2048 , 28 , 1)
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(28, 2048, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(2048, 1024, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(1024, 512, 2),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256,  3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
            
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    
def data_loader(environment_list , N = 101, NP = 201):
    
    
    autoencoder = Autoencoder()
    Q = autoencoder.encoder
    Q.load_state_dict(torch.load(r"/home/sj/Deep-RRT-Star-Implementation/weights/cae_encoder2.pkl"))
    if torch.cuda.is_available():
        Q = Q.to(torch.device("cuda"))

    encoded_w_m=np.zeros((N,28),dtype=np.float32)
    index = 0
    for env in environment_list :
        env = int(env)
#         print('env')
#         print(env)
#         print(image_number)
        image= cv2.imread(f'/home/sj/Deep-RRT-Star-Implementation/images/{env}.jpg' , cv2.IMREAD_GRAYSCALE)
        to_tensor = transforms.ToTensor()
        torch_img = to_tensor(image)
        if torch.cuda.is_available():
            torch_img = torch_img.to(torch.device("cuda"))  # Move input tensor to GPU

        output=Q(torch_img)
        output = output.squeeze()
        output=output.data.cpu()
        output = output.numpy()
        encoded_w_m[index] = output
        index = index + 1
    
    
    file_path = r"/home/sj/Deep-RRT-Star-Implementation/output.csv"
    dtypes = {col: 'float32' for col in pd.read_csv(file_path, nrows=1).columns}
    example = pd.read_csv(file_path, dtype=dtypes)
    #print(len(example))
    index1 = 0
    index2 = 0
    index3 = 1
    path_lengths=np.zeros((N,NP + 1),dtype=np.float32)
    prev_image_number = 0
    max_length=0
    while index1 < len(example) :
        row = example.iloc[index1]
        index_start = row['index_start']
        index_goal = row['index_goal']
        len_path = (index_goal - index_start) + 1
        image_number = row['image_number']
        index1 = index_goal + 1
        index1 = int(index1)
#         print('image number')
#         print(image_number)
        if (image_number == prev_image_number):
            path_lengths[index2][0] = image_number
            path_lengths[index2][index3] = len_path
            index3 = index3 + 1
        else :
#             print('image number')
#             print(prev_image_number)
#             print(index3 - 1)
            index3 = 1
            index2 = index2 + 1
            path_lengths[index2][0] = image_number
            path_lengths[index2][index3] = len_path
            prev_image_number = image_number
            index3 = index3 + 1    
        if len_path > max_length :
            max_length = len_path
            
    
    max_length = int(max_length)
    
    
    path_lengths = path_lengths[:, 1:]
    
    
    
    paths=np.zeros((N,NP,max_length,2), dtype=np.float32)
    index = 0
    index1 = 0
    index2 = 0
    index3 = 0
    prev_image_number = 0
    while(index < len(example)) :
        row = example.iloc[index]
        # index_start = int(row['index_start'])
        # index_goal = int(row['index_goal'])
        index_start = X[index][0]
        image_number = X[index][1]
        index = index_goal + 1
        if (image_number == prev_image_number):
            index3 = 0
            for i in range(index_start , index_goal + 1):
                row = example.iloc[i]
                # x = row['current_x']
                # y = row['current_y']
                x = X[i][0]
                y = X[i][1]
                #print([x , y])
                paths[index1][index2][index3] = [x , y]
                #paths[index1][index2][index3][1] = y
                index3 = index3 + 1
            index2 = index2 + 1  
        else :
            index1 = index1 + 1
            index2 = 0
            index3 = 0
            for i in range(index_start , index_goal + 1):
                row = example.iloc[i]
                # x = row['current_x']
                # y = row['current_y']
                x = X[i][0]
                y = X[i][1]
                #print([x , y])
                paths[index1][index2][index3] = [x , y]
                #paths[index1][index2][index3][1] = y
                index3 = index3 + 1
            prev_image_number = image_number    
            index2 = index2 + 1
    
    
    
    
    dataset=[]
    targets=[]
    for i in range(N):
        for j in range(NP):
            if path_lengths[i][j] > 0 :
                for m in range(0, int(path_lengths[i][j]-1)):
                    data=np.zeros(32,dtype=np.float32)
                    for k in range(0 , 28):
                        data[k] = encoded_w_m[i][k]
                    data[28] = paths[i][j][m][0]
                    data[29] = paths[i][j][m][1]
                    data[30] = paths[i][j][int(path_lengths[i][j]) - 1][0]
                    data[31] = paths[i][j][int(path_lengths[i][j]) - 1][1]
                    
                    temp = [round(paths[i][j][m+1][0] , 6) , round(paths[i][j][m+1][1] , 6)]
                    temp = np.array(temp)
                    targets.append(temp)
                    dataset.append(data)
                    
    data = list(zip(dataset, targets))
    random.shuffle(data)
    dataset,targets=zip(*data)
    
    return np.asarray(dataset),np.asarray(targets)
    
    
file_path = r"/home/sj/Deep-RRT-Star-Implementation/output.csv"
# dtypes = {col: 'float32' for col in pd.read_csv(file_path, nrows=1).columns}
# example = pd.read_csv(file_path, dtype=dtypes)
# index1 = 0
# index3 = 1
# prev_image_number = 0
# environment_list = [0]
# print(len(example))
# while index1 < len(example) :
#     row = example.iloc[index1]
#     index_start = row['index_start']
#     index_goal = row['index_goal']
#     image_number = row['image_number']
#     index1 = index_goal + 1
#     print(index_goal)
#     index1 = int(index1)
#     if (image_number == prev_image_number):
#         index3 = index3 + 1
#     else :
#         environment_list.append(prev_image_number)
#         index3 = 1
#         prev_image_number = image_number
#         index3 = index3 + 1

with open(file_path, 'r') as file2:
    prev_image_number = 0
    environment_list = [0]
    reader = csv.reader(file2)
    for row in reader:
        if row[-1] == 'image_number':
            continue
        if len(row)==5:
            environment_number =  int(row[-1])
            if environment_number != prev_image_number:
                environment_list.append(environment_number)
                prev_image_number = environment_number
print(len(environment_list))

dtypes = {col: 'float32' for col in pd.read_csv(file_path, nrows=1).columns}
example = pd.read_csv(file_path, dtype=dtypes)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = example.iloc[:, :2].values
X = sc.fit_transform(X)

dataset , targets = data_loader(environment_list,101,201)


# print(max_length)

101


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn.utils.rnn import pack_padded_sequence
from torch.autograd import Variable

class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.fc = nn.Sequential(
		nn.Linear(input_size, 1280),nn.PReLU(),nn.Dropout(),
		nn.Linear(1280, 1024),nn.PReLU(),nn.Dropout(),
		nn.Linear(1024, 896),nn.PReLU(),nn.Dropout(),
		nn.Linear(896, 768),nn.PReLU(),nn.Dropout(),
		nn.Linear(768, 512),nn.PReLU(),nn.Dropout(),
		nn.Linear(512, 384),nn.PReLU(),nn.Dropout(),
		nn.Linear(384, 256),nn.PReLU(), nn.Dropout(),
		nn.Linear(256, 256),nn.PReLU(), nn.Dropout(),
		nn.Linear(256, 128),nn.PReLU(), nn.Dropout(),
		nn.Linear(128, 64),nn.PReLU(), nn.Dropout(),
		nn.Linear(64, 32),nn.PReLU(),
		nn.Linear(32, output_size))
        

    def forward(self, x):
        out = self.fc(x)
        return out

In [8]:
def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.to(torch.device("cuda"))
    return x

def get_input(i,data,targets,bs):
    if i+bs<len(data):
        bi=data[i:i+bs]
        bt=targets[i:i+bs]
    else:
        bi=data[i:]
        bt=targets[i:]
    return torch.from_numpy(bi),torch.from_numpy(bt)

#dataset,targets= load_dataset()
#I have to yet write the architecture for the model
mlp = MLP(32, 2)

if torch.cuda.is_available():
    mlp = mlp.to(torch.device("cuda"))
    
criterion = nn.MSELoss()
optimizer = torch.optim.Adagrad(mlp.parameters())

total_loss=[]
sm=100 
#training the model for 1000 epochs
for epoch in range(1000):
    #print ("epoch" + str(epoch))
    avg_loss=0
    for i in range (0,len(dataset),100):
        mlp.zero_grad()
        bi,bt= get_input(i,dataset,targets,100)
        bi=to_var(bi)
        bt=to_var(bt)
        bo = mlp(bi)
        loss = criterion(bo,bt)
        #avg_loss=avg_loss+loss.data[0]
        loss.backward()
        optimizer.step()
        
    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')  
#     print "--average loss:"
#     print avg_loss/(len(dataset)/args.batch_size)
    total_loss.append(avg_loss/(len(dataset)/100))
    # Save the models
    if epoch==sm:
        model_path='model'+str(sm)+'.pkl'
        save_path = '/home/sj/Deep-RRT-Star-Implementation/weights/model_weights'
        torch.save(mlp.state_dict(),os.path.join(save_path , model_path))
        sm=sm+50 # save model after every 50 epochs from 100 epoch ownwards

            
#Following are the commands for saving the final model.            
model_path='/home/sj/Deep-RRT-Star-Implementation/weights/model_weights/final_model.pkl'
torch.save(mlp.state_dict() , model_path)
        


Epoch:1, Loss:608.8848
Epoch:2, Loss:508.3774
Epoch:3, Loss:530.8115
Epoch:4, Loss:522.7702
Epoch:5, Loss:459.7539
Epoch:6, Loss:453.1690
Epoch:7, Loss:502.8882
Epoch:8, Loss:449.5192
Epoch:9, Loss:600.1609
Epoch:10, Loss:462.8765
Epoch:11, Loss:518.3183
Epoch:12, Loss:480.0344
Epoch:13, Loss:536.0453
Epoch:14, Loss:613.7839
Epoch:15, Loss:505.6393
Epoch:16, Loss:470.3039
Epoch:17, Loss:430.5031
Epoch:18, Loss:466.5578
Epoch:19, Loss:436.5826
Epoch:20, Loss:580.5089
Epoch:21, Loss:478.7358
Epoch:22, Loss:541.2933
Epoch:23, Loss:428.0184
Epoch:24, Loss:462.7204
Epoch:25, Loss:446.7130
Epoch:26, Loss:422.9966
Epoch:27, Loss:446.8363
Epoch:28, Loss:519.1493
Epoch:29, Loss:388.6898
Epoch:30, Loss:437.0158
Epoch:31, Loss:404.4762
Epoch:32, Loss:504.2092
Epoch:33, Loss:469.0862
Epoch:34, Loss:481.3764
Epoch:35, Loss:518.8210
Epoch:36, Loss:496.2478
Epoch:37, Loss:479.6609
Epoch:38, Loss:443.1186
Epoch:39, Loss:418.0693
Epoch:40, Loss:528.8066
Epoch:41, Loss:448.8656
Epoch:42, Loss:463.4168
E

KeyboardInterrupt: 

In [8]:
file_path = r"/home/sj/Deep-RRT-Star-Implementation/output.csv"
# dtypes = {col: 'float32' for col in pd.read_csv(file_path, nrows=1).columns}
example = pd.read_csv(file_path, dtype=dtypes)
#print(len(example))
index1 = 0
# index2 = 0
# index3 = 1
# path_lengths=np.zeros((N,NP + 1),dtype=np.float32)
prev_image_number = 0
# max_length=0
number_of_environments = 0
while index1 < len(example) :
    row = example.iloc[index1]
    index_start = row['index_start']
    index_goal = row['index_goal']
#     len_path = (index_goal - index_start) + 1
    image_number = row['image_number']
    #print(index_start)
    index1 = index_goal + 1
    index1 = int(index1)
    if (image_number != prev_image_number):
#         path_lengths[index2][0] = image_number
#         path_lengths[index2][index3] = len_path
#         index3 = index3 + 1
          print('image number')
          print(prev_image_number)
          print(index3 - 1)
          prev_image_number = image_number
#           number_of_environments = number_of_environments + 1
#         index3 = 1
#         index2 = index2 + 1
#         path_lengths[index2][0] = image_number
#         path_lengths[index2][index3] = len_path
#         prev_image_number = image_number
#         index3 = index3 + 1    
#     if len_path > max_length :
#         max_length = len_path

print(number_of_environments)

ValueError: cannot convert float NaN to integer

In [7]:
file_path = r"/home/sj/Deep-RRT-Star-Implementation/output.csv"
dtypes = {col: 'float32' for col in pd.read_csv(file_path, nrows=1).columns}
example = pd.read_csv(file_path, dtype=dtypes)
#print(len(example))
index1 = 0
#     index2 = 0
index3 = 1
#     path_lengths=np.zeros((N,NP + 1),dtype=np.float32)
prev_image_number = 0
#     max_length=0
environment_list = [0]
while index1 < len(example) :
    row = example.iloc[index1]
    index_start = row['index_start']
    index_goal = row['index_goal']
    len_path = (index_goal - index_start) + 1
    image_number = row['image_number']
    print(index_start, ",", index_goal)
    index1 = index_goal + 1
    index1 = int(index1)
    if (image_number == prev_image_number):
#             path_lengths[index2][0] = image_number
#             path_lengths[index2][index3] = len_path
        index3 = index3 + 1
    else :
#         print('image number')
#         print(prev_image_number)
        environment_list.append(image_number)
#         print(index3 - 1)
        index3 = 1
#             index2 = index2 + 1
#             path_lengths[index2][0] = image_number
#             path_lengths[index2][index3] = len_path
        prev_image_number = image_number
        index3 = index3 + 1    
#         if len_path > max_length :
#             max_length = len_path

0.0 , 35.0
36.0 , 62.0
63.0 , 101.0
102.0 , 113.0
114.0 , 134.0
135.0 , 141.0
143.0 , 162.0
nan , nan


ValueError: cannot convert float NaN to integer

In [33]:
print(len(environment_list))

101


In [18]:
import os
import numpy as np
import cv2
directory = r"C:\Users\Navdeep\Downloads\images\images"
for env in environment_list :
    env = np.float32(env)
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".JPG"):
            numeric_part = os.path.splitext(filename)[0]
            numeric_value = np.float32(numeric_part)
            if (numeric_value == env):
                image_path = os.path.join(directory, filename)
                image = cv2.imread(image_path)
                output_directory = r'C:\images_modified'
                output_path = os.path.join(output_directory, filename)
                cv2.imwrite(output_path, image)
            
                        

In [20]:
directory = 'C:\images_modified'
files = os.listdir(directory)
num_files = len(files)
print(num_files)

100


In [35]:
file_path = r"C:\Users\Navdeep\Downloads\output.csv"
dtypes = {col: 'float32' for col in pd.read_csv(file_path, nrows=1).columns}
example = pd.read_csv(file_path, dtype=dtypes)

print(len(example))

317938
